In [58]:
#!pip install "gymnasium[box2d]"
!pip install Box2D


In [59]:
import gymnasium as gym # Use gymnasium instead of gym for numpy compatibility
import numpy as np # numerical operations
import random # random sampling
from collections import deque # replay memory
import tensorflow as tf # tensorflow for neural network
from tensorflow.keras import models, layers, optimizers
import Box2D

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [60]:
class DQNAgent:
  def __init__(self, state_size, action_size):
    # store dimensions and initialize replay memory
    self.state_size = state_size
    self.action_size = action_size
    self.memory = deque(maxlen=2000) # replay memory
    self.gamma = 0.99 # discount rate
    self.epsilon = 1.0 # exploration rate
    self.epsilon_min = 0.01
    self.epsilon_decay = 0.995
    self.learning_rate = 0.001
    # build main and target networks
    self.model = self.build_model()
    self.target_model = self.build_model()
    self.update_target_model()

  def build_model(self):
    model = models.Sequential()
    model.add(layers.Dense(64, input_dim=self.state_size, activation='relu'))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(self.action_size, activation='linear'))
    model.compile(loss='mse', optimizer=optimizers.Adam(learning_rate=self.learning_rate))
    return model

  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return random.randrange(self.action_size)
    act_values = self.model.predict(state, verbose=0)
    return np.argmax(act_values[0])

  def replay(self, batch_size):
    minibatch = random.sample(self.memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
      target = self.model.predict(state, verbose=0)
      if done:
        target[0][action] = reward
      else:
        t = self.target_model.predict(next_state, verbose=0)
        target[0][action] = reward + self.gamma * np.amax(t[0])
      self.model.fit(state, target, epochs=1, verbose=0)

    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay

  def update_target_model(self):
    self.target_model.set_weights(self.model.get_weights())

In [62]:
env = gym.make("LunarLander-v3")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
episodes = 300
batch_size = 64

for e in range(episodes):
  state, _ = env.reset() # Unpack observation and info
  state = np.reshape(state,[1, state_size])
  total_reward = 0

  for time in range(500):
    action = agent.act(state)
    next_state, reward, done, _, _ = env.step(action) # Correctly unpack 5 values
    next_state = np.reshape(next_state, [1, state_size])
    agent.remember(state, action, reward, next_state, done)
    state = next_state
    total_reward += reward
    if done:
      agent.update_target_model()
      print(f"Episodes: {e+1}/{episodes}, Score: {total_reward}, Epsilon: {agent.epsilon:.2f}")
      break
    if len(agent.memory) > batch_size:
      agent.replay (batch_size)